### NLP

- Features we will be using out of the collected data to build knn model:
    1. Nutrition
    2. Tags
    3. Ingredients
- In order to process the tags and ingredients we have to convert them into numerical representations
- For tags it is better to use embeddings as these words are more categorical which have repetitions throughout recipies.
- For ingredients we will be using the tf-idf vectorization 

#### Pre-requisites

In [2]:
import warnings
warnings.filterwarnings(action="ignore")

#### Import dependencies

In [3]:
import pandas as pd
import numpy as np
import re


#### Get the data

In [4]:
df = pd.read_csv("../Data Cleaning/all_combined_clean_data.csv")

In [5]:
df["nutrition"]

0       {'Energy': '195 cal', 'Protein': '10.3 g', 'Ca...
1       {'Energy': '74 cal', 'Protein': '2.6 g', 'Carb...
2       {'Energy': '374 cal', 'Protein': '13.3 g', 'Ca...
3       {'Energy': '92 cal', 'Protein': '0.1 g', 'Carb...
4       {'Energy': '68 cal', 'Protein': '2.3 g', 'Carb...
                              ...                        
3614    {'Calories ': '495 cal', 'Kilojoules ': '2070 ...
3615    {'Calories ': '546 cal', 'Kilojoules ': '2280 ...
3616    {'Calories ': '380 cal', 'Kilojoules ': '1590 ...
3617                                                   {}
3618    {'Calories ': '449 cal', 'Kilojoules ': '1880 ...
Name: nutrition, Length: 3619, dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3619 entries, 0 to 3618
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    3619 non-null   int64 
 1   name          3619 non-null   object
 2   ingredients   3619 non-null   object
 3   instructions  3619 non-null   object
 4   nutrition     3619 non-null   object
 5   time          3607 non-null   object
 6   serving_size  3594 non-null   object
 7   tags          3619 non-null   object
dtypes: int64(1), object(7)
memory usage: 226.3+ KB


### Extracting Features

#### I. Nutrition

- First we need the nutrition values in per recipe so let's explode them

In [7]:
eval(df["nutrition"][0])  #7 different values

{'Energy': '195 cal',
 'Protein': '10.3 g',
 'Carbohydrates': '30.5 g',
 'Fiber': '7.9 g',
 'Fat': '4.1 g',
 'Cholesterol': '0 mg',
 'Sodium': '8.8 mg'}

In [8]:
eval(df["nutrition"].iloc[-1]) #11 different values

{'Calories ': '449 cal',
 'Kilojoules ': '1880 kJ',
 'Protein ': '38 g',
 'Total fat ': '17 g',
 'Saturated fat ': '6 g',
 'Carbohydrates ': '35 g',
 'Sugar ': '8 g',
 'Dietary fibre ': '5 g',
 'Sodium ': '530 mg',
 'Calcium ': '130 mg',
 'Iron ': '3.5 mg'}

In [9]:
eval(df["nutrition"][1960]) #8 different values

{'kcal': '563 g',
 'fat': '26 g',
 'saturates': '3.8 g',
 'carbs': '65 g',
 'sugars': '9 g',
 'fibre': '8 g',
 'protein': '17 g',
 'salt': '0.6 g'}

- Since our data is collected from three different sources, there seems to be varying measures of nutrition, let us keep the most basic nutrition values and drop the rest for now. 
- Later, we could also impute the additional nutrients like calcium, iron using a food nutrition dataset

In [10]:
#feature set
features = df[["ingredients","nutrition","tags"]]

In [11]:
features.head(3)

,ingredients,nutrition,tags
0,['2 cups sprouted vaal (field beans/ butter be...,"{'Energy': '195 cal', 'Protein': '10.3 g', 'Ca...","['Non-stick Pan', 'Boiled Indian recipes', 'Sa..."
1,"['2 1/2 cups capsicum cubes', '1/2 cup low-fat...","{'Energy': '74 cal', 'Protein': '2.6 g', 'Carb...","['Non Stick Kadai Veg', 'Antioxidant Rich Indi..."
2,"['1 cup sliced onions', '3 tbsp roughly choppe...","{'Energy': '374 cal', 'Protein': '13.3 g', 'Ca...","['Non-stick Pan', 'Indian Dinner', 'Indian Lun..."


In [12]:
features["nutrition"]

0       {'Energy': '195 cal', 'Protein': '10.3 g', 'Ca...
1       {'Energy': '74 cal', 'Protein': '2.6 g', 'Carb...
2       {'Energy': '374 cal', 'Protein': '13.3 g', 'Ca...
3       {'Energy': '92 cal', 'Protein': '0.1 g', 'Carb...
4       {'Energy': '68 cal', 'Protein': '2.3 g', 'Carb...
                              ...                        
3614    {'Calories ': '495 cal', 'Kilojoules ': '2070 ...
3615    {'Calories ': '546 cal', 'Kilojoules ': '2280 ...
3616    {'Calories ': '380 cal', 'Kilojoules ': '1590 ...
3617                                                   {}
3618    {'Calories ': '449 cal', 'Kilojoules ': '1880 ...
Name: nutrition, Length: 3619, dtype: object

In [13]:
# apply a function to this in order to convert values from string to numeric and 
# remove the units like 'cal' and 'g'
# nutrients we are gonna keep: 
# ['Calories', 'Protein', 'Carbohydrates', 'Fiber', 'Fat', 'Sodium']

import ast

def nutrition_preprocessing(nutrients_series):
    mapped = {"calories": None, "protein":None, "carbohydrates":None, "fiber":None, "fat":None, "sodium":None}
    to_dict = ast.literal_eval(nutrients_series)
    for key,value in to_dict.items():
        prep_key = key.strip().lower()
        cleaned = re.sub(r'[^\d\.]', '', value)
        if cleaned == "":  # Some values in carbohydrates are like "N/A"
            continue
        if prep_key in ["calories","energy","kcal"]:
            mapped["calories"]= float(cleaned)
        if prep_key == "protein":
            mapped["protein"]= float(cleaned)
        if prep_key in ["carbohydrates","carbs"]:
            mapped["carbohydrates"]= float(cleaned)
        if prep_key in ["fiber","dietary fibre","fibre"]:
            mapped["fiber"]= float(cleaned)    
        if prep_key in ["fat","total fat"]:
            mapped["fat"]= float(cleaned)
        if prep_key in ["sodium","salt"]:
            mapped["sodium"]= float(cleaned)
    return mapped

In [14]:
cleaned = features["nutrition"].apply(nutrition_preprocessing)

In [15]:
exploded_df = pd.json_normalize(cleaned)

In [16]:
exploded_df

,calories,protein,carbohydrates,fiber,fat,sodium
0,195.0,10.3,30.5,7.9,4.1,8.8
1,74.0,2.6,9.5,3.2,2.9,21.8
2,374.0,13.3,16.4,4.2,28.4,98.6
3,92.0,0.1,0.4,0.0,10.0,0.0
4,68.0,2.3,3.9,3.4,5.3,114.5
...,...,...,...,...,...,...
3614,495.0,19.0,45.0,14.0,25.0,650.0
3615,546.0,23.0,50.0,12.0,28.0,770.0
3616,380.0,16.0,55.0,11.0,9.0,350.0
3617,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# add the exploded column to features 
features_exploded = features.drop("nutrition", axis=1).join(exploded_df)

In [18]:
features_exploded.head(3)

,ingredients,tags,calories,protein,carbohydrates,fiber,fat,sodium
0,['2 cups sprouted vaal (field beans/ butter be...,"['Non-stick Pan', 'Boiled Indian recipes', 'Sa...",195.0,10.3,30.5,7.9,4.1,8.8
1,"['2 1/2 cups capsicum cubes', '1/2 cup low-fat...","['Non Stick Kadai Veg', 'Antioxidant Rich Indi...",74.0,2.6,9.5,3.2,2.9,21.8
2,"['1 cup sliced onions', '3 tbsp roughly choppe...","['Non-stick Pan', 'Indian Dinner', 'Indian Lun...",374.0,13.3,16.4,4.2,28.4,98.6


In [19]:
# droping the two na values because of missing carbohydrate found while extracting 
# nutrients
features_exploded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3619 entries, 0 to 3618
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ingredients    3619 non-null   object 
 1   tags           3619 non-null   object 
 2   calories       3581 non-null   float64
 3   protein        3582 non-null   float64
 4   carbohydrates  3577 non-null   float64
 5   fiber          3581 non-null   float64
 6   fat            3582 non-null   float64
 7   sodium         3566 non-null   float64
dtypes: float64(6), object(2)
memory usage: 226.3+ KB


In [20]:
features_exploded[features_exploded.isnull().any(axis=1)].tail(3)

,ingredients,tags,calories,protein,carbohydrates,fiber,fat,sodium
3606,['1 large slice sourdough or wholegrain bread'...,"['Gluten-free option', 'Ready in 20 minutes', ...",NaN,NaN,NaN,NaN,NaN,NaN
3611,"['spray oil', '1 small potato, diced', '1 cup ...","['Gluten-free option', 'Nut free', 'Meals for ...",NaN,NaN,NaN,NaN,NaN,NaN
3617,"['1 cup mesclun or other salad leaves', '1 cup...","['Meals for one', 'No, or minimal, cooking', '...",NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# We have no option but to drop these rows as nutrient info for them are also missing

In [22]:
features_exploded.dropna(inplace=True)

In [23]:
features_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3560 entries, 0 to 3618
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ingredients    3560 non-null   object 
 1   tags           3560 non-null   object 
 2   calories       3560 non-null   float64
 3   protein        3560 non-null   float64
 4   carbohydrates  3560 non-null   float64
 5   fiber          3560 non-null   float64
 6   fat            3560 non-null   float64
 7   sodium         3560 non-null   float64
dtypes: float64(6), object(2)
memory usage: 250.3+ KB


In [24]:
nutrients = features_exploded.drop(["ingredients","tags"],axis=1)

In [25]:
nutrients.describe()

,calories,protein,carbohydrates,fiber,fat,sodium
count,3560.000000,3560.000000,3560.000000,3560.000000,3.560000e+03,3560.000000
mean,243.524438,12.660281,26.046096,5.684955,1.091591e+03,210.312846
std,179.831314,13.025845,18.678152,15.025915,6.458652e+04,316.882171
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,94.000000,2.600000,10.800000,1.800000,2.400000e+00,10.575000
50%,194.000000,6.700000,22.150000,4.000000,6.800000e+00,54.950000
75%,390.000000,20.825000,38.800000,8.000000,1.380000e+01,353.000000
max,2167.000000,79.000000,102.800000,680.000000,3.853612e+06,5829.600000


In [26]:
nutrients.keys()

Index(['calories', 'protein', 'carbohydrates', 'fiber', 'fat', 'sodium'], dtype='object')

- Scaling 

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_nutrients = scaler.fit_transform(nutrients)

In [28]:
scaled_nutrients

array([[-0.26987099, -0.18122528,  0.23848881,  0.14743569, -0.0168401 ,
        -0.63601292],
       [-0.9428183 , -0.7724408 , -0.88597738, -0.16540119, -0.01685868,
        -0.59498244],
       [ 0.7256461 ,  0.04911842, -0.51650991, -0.09884015, -0.01646381,
        -0.35258702],
       ...,
       [ 1.68223235,  0.79389641,  1.28263598,  0.42033594, -0.01647   ,
         1.76647925],
       [ 0.75901538,  0.25642776,  1.55036602,  0.3537749 , -0.01676422,
         0.44087926],
       [ 1.14276219,  1.94561494,  0.47944585, -0.04559132, -0.01664034,
         1.00899354]], shape=(3560, 6))

In [29]:
scaled_nutrients_df = pd.DataFrame(scaled_nutrients,columns=nutrients.keys())

In [30]:
scaled_nutrients_df.describe()

,calories,protein,carbohydrates,fiber,fat,sodium
count,3560.000000,3.560000e+03,3.560000e+03,3.560000e+03,3.560000e+03,3.560000e+03
mean,0.000000,-1.277380e-16,1.916070e-16,-6.386901e-17,-2.993860e-18,-3.193450e-17
std,1.000140,1.000140e+00,1.000140e+00,1.000140e+00,1.000140e+00,1.000140e+00
min,-1.354373,-9.720720e-01,-1.394664e+00,-3.783965e-01,-1.690359e-02,-6.637874e-01
25%,-0.831587,-7.724408e-01,-8.163676e-01,-2.585866e-01,-1.686643e-02,-6.304107e-01
50%,-0.275433,-4.576377e-01,-2.086204e-01,-1.121524e-01,-1.679829e-02,-4.903547e-01
75%,0.814631,6.268972e-01,6.829207e-01,1.540918e-01,-1.668990e-02,4.503478e-01
max,10.697502,5.093646e+00,4.109865e+00,4.488311e+01,5.965735e+01,1.773554e+01


#### 2. Ingredients

In [31]:
sample= features_exploded["ingredients"].sample(10)
# let's start with this sample, then we can apply a cleaning function to the whole column

In [32]:
for s in sample:
    print(s)

['2 cups sliced mushrooms (khumbh)', '1 tsp oil', '1 tbsp finely chopped garlic (lehsun)', '1 tsp finely chopped green chillies', 'salt to taste', '1/4 cup finely chopped spring onion greens', '1/2 tsp soy sauce']
['1 1/2 cups soaked and boiled rajma (kidney beans)', '1 1/2 cups soaked and boiled kabuli chana (white chick peas)', '1 cup coloured capsicum cubes', '', '1/2 cup chopped spring onions whites and greens', '1/4 cup chopped coriander (dhania)', '1 tsp finely chopped green chillies', '2 tsp extra virgin olive oil', '1 tbsp lemon juice', '1/2 tsp chilli powder', '1/2 tsp sea salt (khada namak)']
['1 large red onion, thinly sliced', '2 cups apple', 'vinegar', '½ teaspoon sugar', '½ teaspoon salt', '4 large portobello mushrooms, stalk removed', '1 yellow or orange capsicum, thickly sliced', 'spray oil', '1 teaspoon ground cumin', '1 teaspoon ground', '1 teaspoon smoked paprika', '½ onion, finely chopped', '3 cloves garlic, crushed', '400g can no-added-salt cannellini beans, draine

In [33]:
sample.iloc[0]

"['2 cups sliced mushrooms (khumbh)', '1 tsp oil', '1 tbsp finely chopped garlic (lehsun)', '1 tsp finely chopped green chillies', 'salt to taste', '1/4 cup finely chopped spring onion greens', '1/2 tsp soy sauce']"

- Right now are ingredients column contains a list of splitted strings but what we need is a list of tokens
- Problems :
    * removing measures like cups and tbsp
    * removing quantities
    * map english and hindi ingredients (paneer --> cottage cheese)
    * lemon juice, flax seeds should be one token 
    * lemmatization 

1. Tokenization

In [34]:
def custom_tokenizer(text):
    # Remove quantities and units
    text = re.sub(r'\b\d+/?\d*\b', '', text.lower())  # remove quantities
    text = re.sub(r'\b(?:tsp|tbsp|cup|cups|g|gram|grams|kg|ml|l|oz|teaspoon|tablespoon|pinch)\b', '', text, flags=re.IGNORECASE)

    # Tokenize 
    tokens = re.findall(r'\b[a-zA-Z]+\b', text)
    return tokens 

In [35]:
custom_tokenizer(sample.iloc[0])

['sliced',
 'mushrooms',
 'khumbh',
 'oil',
 'finely',
 'chopped',
 'garlic',
 'lehsun',
 'finely',
 'chopped',
 'green',
 'chillies',
 'salt',
 'to',
 'taste',
 'finely',
 'chopped',
 'spring',
 'onion',
 'greens',
 'soy',
 'sauce']

In [36]:
for s in sample:
    print(custom_tokenizer(s))

['sliced', 'mushrooms', 'khumbh', 'oil', 'finely', 'chopped', 'garlic', 'lehsun', 'finely', 'chopped', 'green', 'chillies', 'salt', 'to', 'taste', 'finely', 'chopped', 'spring', 'onion', 'greens', 'soy', 'sauce']
['soaked', 'and', 'boiled', 'rajma', 'kidney', 'beans', 'soaked', 'and', 'boiled', 'kabuli', 'chana', 'white', 'chick', 'peas', 'coloured', 'capsicum', 'cubes', 'chopped', 'spring', 'onions', 'whites', 'and', 'greens', 'chopped', 'coriander', 'dhania', 'finely', 'chopped', 'green', 'chillies', 'extra', 'virgin', 'olive', 'oil', 'lemon', 'juice', 'chilli', 'powder', 'sea', 'salt', 'khada', 'namak']
['large', 'red', 'onion', 'thinly', 'sliced', 'apple', 'vinegar', 'sugar', 'salt', 'large', 'portobello', 'mushrooms', 'stalk', 'removed', 'yellow', 'or', 'orange', 'capsicum', 'thickly', 'sliced', 'spray', 'oil', 'ground', 'cumin', 'ground', 'smoked', 'paprika', 'onion', 'finely', 'chopped', 'cloves', 'garlic', 'crushed', 'can', 'no', 'added', 'salt', 'cannellini', 'beans', 'drained

* Observations:
    - Stop words still need to be removed 
    - Have to remove verbs like added, chopped, splash, peeled (nltk pos tag)
    - Adverbs like roughly 
    - there are some ingredients that are mentioned both in hindi and english like: 'flour' and 'gehun ka atta', "panner" and "cottage cheeze" (ignore hindi words by filtering from english dictionary)
    - might need to specify n grams as there are some tokens with more than one word so order also matters like "gehun ka atta", "baby spinach" "cottage cheeze", "chocolate chips" (however this will increase vector size)


In [37]:
# pip install spacy
# python -m spacy download en_core_web_sm


In [38]:
import spacy
# load english model 
nlp = spacy.load('en_core_web_sm')

In [39]:
doc = nlp(sample.iloc[0])

In [40]:
type(doc) #this is a spacy object containing tokens

spacy.tokens.doc.Doc

In [41]:
for token in doc:
    print(f"{token.text:<15} | POS: {token.pos_:<10} | Lemma: {token.lemma_}")

[               | POS: X          | Lemma: [
'               | POS: NUM        | Lemma: '
2               | POS: NUM        | Lemma: 2
cups            | POS: NOUN       | Lemma: cup
sliced          | POS: VERB       | Lemma: slice
mushrooms       | POS: NOUN       | Lemma: mushroom
(               | POS: PUNCT      | Lemma: (
khumbh          | POS: NOUN       | Lemma: khumbh
)               | POS: PUNCT      | Lemma: )
'               | POS: PUNCT      | Lemma: '
,               | POS: PUNCT      | Lemma: ,
'               | POS: PUNCT      | Lemma: '
1               | POS: NUM        | Lemma: 1
tsp             | POS: NOUN       | Lemma: tsp
oil             | POS: NOUN       | Lemma: oil
'               | POS: PUNCT      | Lemma: '
,               | POS: PUNCT      | Lemma: ,
'               | POS: PUNCT      | Lemma: '
1               | POS: NUM        | Lemma: 1
tbsp            | POS: NOUN       | Lemma: tbsp
finely          | POS: ADV        | Lemma: finely
chopped         | POS: VE

In [42]:
def custom_tokenizer(text):
    #text cleaning
    text = text.lower()
     # Replace unicode superscript fractions and other weird chars
    text = re.sub(r'[¼½¾⅓⅔⁄¹²³⁴⁵⁶⁷⁸⁹⁰]', ' ', text)

    # Separate numbers stuck to units or words (e.g., "100ml" → "100 ml")
    text = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', text)

    # Remove standalone numbers and units
    text = re.sub(r'\b\d+/?\d*\b', ' ', text)
    text = re.sub(r'\b(?:ml|l|tsp|tbsp|cup|cups|g|kg|oz|gram|grams|pinch|cm|inch|store|style)\b', '', text)

    # Remove hyphens or leading punctuation leftovers
    text = re.sub(r'^[^\w\s]+|[^\w\s]+$', '', text)

    # Remove extra symbols, keep only words
    text = re.sub(r'[^\w\s]', ' ', text)

    # Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # spacy to remove stop words and anythings except NOUN and PROPN
    # return lemmatized tokens
    text = nlp(text)
    tokens = [token.lemma_ for token in text 
              if not token.is_stop 
              and not token.is_punct 
              and token.pos_ in {'NOUN','PROPN'}]
    
    return tokens 

In [43]:
for s in sample:
    print(custom_tokenizer(s))

['mushroom', 'khumbh', 'oil', 'lehsun', 'chilli', 'salt', 'spring', 'onion', 'green', 'soy', 'sauce']
['rajma', 'kidney', 'bean', 'kabuli', 'chana', 'white', 'chick', 'peas', 'capsicum', 'cube', 'spring', 'onion', 'white', 'green', 'coriander', 'dhania', 'chilli', 'virgin', 'olive', 'oil', 'lemon', 'juice', 'chilli', 'powder', 'sea', 'salt', 'khada', 'namak']
['onion', 'apple', 'vinegar', 'teaspoon', 'sugar', 'teaspoon', 'salt', 'mushroom', 'stalk', 'capsicum', 'spray', 'oil', 'teaspoon', 'ground', 'cumin', 'teaspoon', 'ground', 'teaspoon', 'paprika', 'onion', 'clove', 'garlic', 'salt', 'cannellini', 'bean', 'teaspoon', 'oregano', 'ground', 'pepper', 'teaspoon', 'chilli', 'cheddar', 'cheese', 'cream', 'wholegrain', 'burger', 'bun', 'baby', 'lettuce', 'tomato', 'chilli', 'optional']
['cabbage', 'fenugreek', 'methi', 'jowar', 'white', 'millet', 'flour', 'wheat', 'flour', 'gehun', 'ka', 'atta', 'besan', 'bengal', 'flour', 'chilli', 'lemon', 'juice', 'salt', 'oil', 'cooking']
['tablespoon'

- apply it to our dataset now

In [44]:
features_exploded["ingredients"].apply(custom_tokenizer)

0       [field, bean, butter, bean, oil, cumin, jeera,...
1       [capsicum, cube, paneer, cottage, cheese, coco...
2       [onion, cashew, nut, kaju, chilli, cottage, ch...
3       [oil, vinegar, mustard, sarson, powder, salt, ...
4                        [flax, seed, lemon, juice, salt]
                              ...                        
3613    [spray, oil, red, cabbage, cut, wedge, bulb, f...
3614    [onion, apple, vinegar, teaspoon, sugar, teasp...
3615    [yoghurt, tablespoon, paste, tablespoon, lime,...
3616    [carrot, desiree, potato, pea, capsicum, sprin...
3618    [spray, oil, tablespoon, olive, oil, boneless,...
Name: ingredients, Length: 3560, dtype: object

VECTORIZING USING tfidfVectorizer
- use min_df for filtering out rare words that don't generalize or add value (e.g., spelling errors, unique names, uncommon variants).
- use max_features to limit feature space to prevent sparsity, overfitting

Edit -- Without these hyperparameters the feature set or vocabulary was coming out to be 1576, it was also considering features with spelling errors, possibly missed during the cleaning

In [45]:
# Convert this into a vector
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, min_df= 5, max_features= 800)
#Edit: ngrams is causing too much of array size (18652 features), 
#      this is more than the number of rows in our set, for now, lets take it without ngrams
tfidf_matrix = vectorizer.fit_transform(features_exploded["ingredients"])

In [78]:
tfidf_matrix.toarray()[0]

array([0.17614604, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.18363797, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.3100418 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [47]:
# Get feature (token) names
feature_names = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

df_tfidf.head()  



,adrak,agria,ajmoda,ajwain,akhrot,alfalfa,allspice,almond,alsi,amaranth,...,x,xa0,yam,yeast,yellow,yoghurt,yogurt,yolk,zest,zucchini
0,0.176146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.173541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.182461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
feature_names

array(['adrak', 'agria', 'ajmoda', 'ajwain', 'akhrot', 'alfalfa',
       'allspice', 'almond', 'alsi', 'amaranth', 'amchur', 'amla', 'anar',
       'anardana', 'anchovy', 'anise', 'anjeer', 'apple', 'approx',
       'apricot', 'arborio', 'arhar', 'aril', 'asafoetida', 'asparagus',
       'atta', 'avocado', 'baby', 'bacon', 'badam', 'badi', 'bag',
       'baguette', 'baingan', 'bajra', 'baking', 'ball', 'banana',
       'barbecue', 'barley', 'basil', 'basis', 'basmati', 'baton',
       'batter', 'bay', 'bean', 'beans', 'beansprout', 'beef', 'beej',
       'beet', 'beetroot', 'bengal', 'berry', 'besan', 'bhaji', 'bhindi',
       'bhopla', 'bird', 'biscuit', 'bite', 'black', 'blackberry',
       'blanched', 'blend', 'blueberry', 'boiling', 'bok', 'bone',
       'boneless', 'bottle', 'bran', 'bread', 'breadcrumb', 'breast',
       'brine', 'brinjal', 'broccoli', 'broccolini', 'brown', 'brushing',
       'brussels', 'buckwheat', 'bulb', 'bun', 'bunch', 'bunche',
       'bunches', 'burger', 

In [49]:
#final features 
# df_tfidf = df_tfidf.reset_index(drop=True)
# scaled_nutrients_df = scaled_nutrients_df.reset_index(drop=True)
X = scaled_nutrients_df.join(df_tfidf, rsuffix="_tfidf")
X.head()

,calories,protein,carbohydrates,fiber,fat,sodium,adrak,agria,ajmoda,ajwain,...,x,xa0,yam,yeast,yellow,yoghurt,yogurt,yolk,zest,zucchini
0,-0.269871,-0.181225,0.238489,0.147436,-0.016840,-0.636013,0.176146,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.942818,-0.772441,-0.885977,-0.165401,-0.016859,-0.594982,0.173541,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.725646,0.049118,-0.516510,-0.098840,-0.016464,-0.352587,0.182461,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.842710,-0.964394,-1.373246,-0.378397,-0.016749,-0.663787,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.976188,-0.795475,-1.185835,-0.152089,-0.016822,-0.302404,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, we have all the features names ready, let's train the model in knn

### Model building: Finding nearest neighbours through KNN

In [50]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X)

NearestNeighbors(metric='euclidean')

In [51]:
# Let's say we have the input features like :
input_nutrients ={
    'calories': 345.8,
    'protein': 18.2,
    'carbohydrates': 63.7,
    'fiber': 5.9,
    'fat': 14.4,
    'sodium': 738.5
}

# available pantry items: 
input_ingredients = "flax seeds olive oil chopped onions garlic paste adrak boiled potatoes green chili cumin powder turmeric fresh coriander lemon juice"



In [52]:
# Now using this, let's find out the most similar recipies from our data, 
# for that we have to convert the ingredients the same way using the vectorizer
input_vectorized_ingredients = vectorizer.transform([input_ingredients])
input_vectorized_ingredients

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14 stored elements and shape (1, 800)>

In [76]:
input_vectorized_ingredients.toarray()

array([[0.29169089, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [53]:
input_ingredients_df = pd.DataFrame(input_vectorized_ingredients.toarray(), columns=feature_names)
input_ingredients_df


,adrak,agria,ajmoda,ajwain,akhrot,alfalfa,allspice,almond,alsi,amaranth,...,x,xa0,yam,yeast,yellow,yoghurt,yogurt,yolk,zest,zucchini
0,0.291691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
list(input_nutrients.values())

[345.8, 18.2, 63.7, 5.9, 14.4, 738.5]

In [55]:
input_nutrients = np.array(list(input_nutrients.values()),ndmin=2)

In [56]:
input_scaled_nutrients = scaler.transform(input_nutrients)

In [73]:
input_scaled_nutrients

array([[ 0.56881044,  0.42534648,  2.0162163 ,  0.01431361, -0.0166806 ,
         1.66705925]])

In [57]:
input_features = pd.DataFrame(input_scaled_nutrients,columns=scaled_nutrients_df.keys())
input_features

,calories,protein,carbohydrates,fiber,fat,sodium
0,0.56881,0.425346,2.016216,0.014314,-0.016681,1.667059


In [58]:
input_features = input_features.join(input_ingredients_df, rsuffix="_tfidf")
input_features

,calories,protein,carbohydrates,fiber,fat,sodium,adrak,agria,ajmoda,ajwain,...,x,xa0,yam,yeast,yellow,yoghurt,yogurt,yolk,zest,zucchini
0,0.56881,0.425346,2.016216,0.014314,-0.016681,1.667059,0.291691,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
distances, indices = knn.kneighbors(input_features)

In [60]:
distances #ordered

array([[1.40583734, 1.42650061, 1.47721682, 1.48233354, 1.49234101]])

In [61]:
indices  #indices from our df

array([[2808, 3238, 3002, 2947, 2232]])

In [62]:
# Top 5 similar recipies
df.iloc[indices.flatten()]

,Unnamed: 0,name,ingredients,instructions,nutrition,time,serving_size,tags
2808,2808,Oregano chicken with warm tomato and olive salsa,"['2 x 250g chicken breast fillets, each cut ho...",['1 Place the chicken on a plate. Sprinkle bot...,"{'Calories ': '503 cal', 'Kilojoules ': '2101 ...",30 mins,4,"['High fibre', 'High protein', '3 vege serves'..."
3238,3238,Asian salmon and super greens oven-steamed bake,"['2 cups small Brussels sprouts, trimmed, thin...","['1 Preheat oven to 180°C. Into a large, deep ...","{'Calories ': '584 cal', 'Kilojoules ': '2440 ...",40 mins,4,"['Dairy free', 'Diabetes-friendly', 'Gluten-fr..."
3002,3002,Miso and corn soup with ramen noodles,"['4 cups reduced-salt vegetable stock', '1 tab...","['1 Combine stock, 2 cups of water and miso pa...","{'Calories ': '266 cal', 'Kilojoules ': '1112 ...",30 mins,4,"['Dairy free', 'Vegetarian', '3 vege serves', ..."
2947,2947,Blueberry muffins made healthier,"['2 medium eggs', '250ml jar apple sauce', '5 ...",['1 Preheat the oven to 190°C/fan 170°C/gas 5 ...,"{'Calories ': '198 cal', 'Kilojoules ': '829 k...",35 mins,12,"['High fibre', 'Family favourites', 'Makeovers']"
2232,2232,Roasted garlic and pumpkin soup,"['¼ cup rolled', '¼ cup cubed grainy bread (st...","['1 To make the savoury granola, preheat the o...","{'Calories ': '417 cal', 'Kilojoules ': '1742 ...",45 mins,4,"['Diabetes-friendly', 'High fibre', 'Low sodiu..."


In [63]:
##### Create a pipeline
def top_5_recipies(cal,protein,carbs,fiber,fat,sodium, ingredients):
    input_nutrients ={
    'calories': cal,
    'protein': protein,
    'carbohydrates': carbs,
    'fiber': fiber,
    'fat': fat,
    'sodium': sodium
}
    input_vectorized_ingredients = vectorizer.transform([ingredients])
    input_ingredients_df = pd.DataFrame(input_vectorized_ingredients.toarray(), columns=feature_names)
    input_nutrients = np.array(list(input_nutrients.values()),ndmin=2)
    input_scaled_nutrients = scaler.transform(input_nutrients)
    input_nutrients_df = pd.DataFrame(input_scaled_nutrients,columns=scaled_nutrients_df.keys())
    input_features = input_nutrients_df.join(input_ingredients_df, rsuffix="_tfidf")
    distances, indices = knn.kneighbors(input_features)
    return df.iloc[indices.flatten()]


In [64]:
result = top_5_recipies(290,10,38,4.5,8,120,"rice coconut dal vegetable")

In [ ]:
result1 = result[['name','ingredients','nutrition']]
for i in result1.to_records():
    print(i)

KeyError: "['nutrients'] not in index"

In [65]:
result['name'].apply(print)

green moong dal khichdi recipe
Slow-cooked pulled beef
cabbage pulao recipe
Mushrooms stuffed with kale, ricotta and seeds
bajra khichdi for acidity recipe


1687    None
3014    None
1513    None
2152    None
1468    None
Name: name, dtype: object

In [66]:
result['ingredients'].apply(print)


['1/2 cup rice (chawal)', '1/2 cup green moong dal (split green gram)', '1 tbsp oil', '1 tsp mustard seeds ( rai / sarson)', '1 tsp urad dal (split black lentils)', '1/4 tsp asafoetida (hing)', '2 tsp finely chopped garlic (lehsun)', '1/2 cup finely chopped onion', '1 tsp chilli powder', '1/4 tsp turmeric powder (haldi)', '1 tsp coriander-cumin seeds (dhania-jeera) powder', 'salt to taste', '1 tbsp finely chopped coriander (dhania)', '3 cups cooked long grained rice']
['1kg beef brisket, fat trimmed', '1 cup passata', '1 tablespoon', '2 tablespoon Worcestershire sauce', '2 teaspoon paprika', '2 cloves garlic, crushed', '6 wholemeal tortillas, warmed', '200g salad leaves', '4 tomatoes, diced', '170g reduced-fat Greek yogurt']
['1 1/2 cups shredded cabbage', '1 1/2 cups soaked and cooked brown rice', '2 tsp coconut oil or', '1/2 tsp mustard seeds ( rai / sarson)', '1/2 tsp urad dal (split black lentils)', '4 to 5 curry leaves (kadi patta)', '2 tbsp raw peanuts', 'a pinch of asafoetida (h

1687    None
3014    None
1513    None
2152    None
1468    None
Name: ingredients, dtype: object

In [67]:
result['nutrition'].apply(print)

{'Energy': '209 cal', 'Protein': '8.4 g', 'Carbohydrates': '34.3 g', 'Fiber': '3.3 g', 'Fat': '4.2 g', 'Cholesterol': '0 mg', 'Sodium': '10.2 mg'}
{'Calories ': '449 cal', 'Kilojoules ': '1879 kJ', 'Protein ': '43 g', 'Total fat ': '15.4 g', 'Saturated fat ': '6.1 g', 'Carbohydrates ': '34.1 g', 'Sugar ': '10.9 g', 'Dietary fibre ': '34.1 g', 'Sodium ': '512 mg', 'Calcium ': '92 mg', 'Iron ': '5.7 mg'}
{'Energy': '218 cal', 'Protein': '6.6 g', 'Carbohydrates': '28.2 g', 'Fiber': '4.9 g', 'Fat': '9 g', 'Cholesterol': '0 mg', 'Sodium': '19.5 mg'}
{'Calories ': '303 cal', 'Kilojoules ': '1268 kJ', 'Protein ': '17.7 g', 'Total fat ': '15.2 g', 'Saturated fat ': '5.1 g', 'Carbohydrates ': '19.8 g', 'Sugar ': '8.1 g', 'Dietary fibre ': '8.3 g', 'Sodium ': '158 mg', 'Calcium ': '234 mg', 'Iron ': '3 mg'}
{'Energy': '252 cal', 'Protein': '11.3 g', 'Carbohydrates': '38.2 g', 'Fiber': '5.8 g', 'Fat': '6.1 g', 'Cholesterol': '0 mg', 'Sodium': '12 mg'}


1687    None
3014    None
1513    None
2152    None
1468    None
Name: nutrition, dtype: object

- the model seems to work decently, however if we have more recipies, it will give better result

In [68]:
# another try 
result = top_5_recipies(290,10,38,4.5,8,120,"yogurt lettuce cheeze spinach")

In [69]:
result['name'].apply(print)

Paneer and Suva Sandwich
Nutritious Burger
Quinoa Paneer Carrot Peppers Salad, for Lunch Or Dinner
Chimichurri
palak bajra khichdi recipe


359     None
420     None
1630    None
2699    None
1847    None
Name: name, dtype: object

In [70]:
result['ingredients'].apply(print)

['8 multigrain bread', '', '4 curly red lettuce or cos lettuce , torn into pieces', 'melted butter for brushing', '1 cup crumbled paneer (cottage cheese)', '1/4 cup finely chopped dill leaves', '2 tsp finely chopped green chillies', 'salt to taste']
['1/2 cup soy granules', '3/4 cup grated carrot', '1/2 cup crumbled low fat paneer (cottage cheese)', '1/2 cup finely chopped onion', '2 tbsp whole wheat flour (gehun ka atta)', 'hummus', '1/2 tsp garlic (lehsun) paste', '1 tsp green chilli paste', '1/4 cup finely chopped mint leaves (phudina)', 'salt and', '1 tsp oil for greasing and cooking', '3/4 cup low fat curds (dahi)', '1/4 cup chopped spring onion greens', '1/2 cup finely chopped capsicum ( chopped red capsicum ,', '1/2 tsp garlic (lehsun) paste', '1 tsp dry red chilli flakes (paprika)', 'salt to taste', '6 burger buns', '1 1/2 tsp butter for cooking', '6 lettuce', '18 sliced cucumber', '18 tomato slices', '6 onion slice']
['1/2 cup coloured capsicum cubes', '1/4 cup paneer (cottage

359     None
420     None
1630    None
2699    None
1847    None
Name: ingredients, dtype: object

In [71]:
result['nutrition'].apply(print)

{'Energy': '261 cal', 'Protein': '9.7 g', 'Carbohydrates': '32.5 g', 'Fiber': '2.2 g', 'Fat': '10.3 g', 'Cholesterol': '0 mg', 'Sodium': '7.2 mg'}
{'Energy': '266 cal', 'Protein': '9.8 g', 'Carbohydrates': '41 g', 'Fiber': '4.2 g', 'Fat': '6.9 g', 'Cholesterol': '1.7 mg', 'Sodium': '72 mg'}
{'Energy': '356 cal', 'Protein': '11.6 g', 'Carbohydrates': '41.1 g', 'Fiber': '10.7 g', 'Fat': '17.7 g', 'Cholesterol': '0 mg', 'Sodium': '44.4 mg'}
{'Calories ': '151 cal', 'Kilojoules ': '632 kJ', 'Protein ': '0.7 g', 'Total fat ': '16.2 g', 'Saturated fat ': '2.5 g', 'Carbohydrates ': '0.7 g', 'Sugar ': '0.3 g', 'Dietary fibre ': '0.9 g', 'Sodium ': '9 mg', 'Calcium ': '30 mg', 'Iron ': '1.1 mg'}
{'Energy': '249 cal', 'Protein': '11.8 g', 'Carbohydrates': '38.8 g', 'Fiber': '6.4 g', 'Fat': '5.2 g', 'Cholesterol': '0 mg', 'Sodium': '25.4 mg'}


359     None
420     None
1630    None
2699    None
1847    None
Name: nutrition, dtype: object

In [72]:
# save the knn model and vectorizors
import pickle 
with open("knn.pkl",'wb') as f:
    pickle.dump(knn,f)
with open("ingredient_vectorizer.pkl",'wb') as g:
    pickle.dump(vectorizer,g)
with open("nutrients_scaler.pkl",'wb') as i:
    pickle.dump(scaler,i)